In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

from pyspark import SparkContext
import pyspark.sql.functions as F
from pyspark.sql.session import SparkSession

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sc = SparkContext('local')
spark = SparkSession(sc)

df = spark.read.csv('/Users/fanyang/Documents/musicbox/data/event1_downsample.csv', header=True).cache()
df

DataFrame[uid: string, event: string, song_id: string, date: string]

In [3]:
df = df.withColumn('date', F.col('date').cast('date'))
df

DataFrame[uid: string, event: string, song_id: string, date: date]

In [4]:
# count distinct 'uid'
df.select('uid').distinct().count()

59436

In [5]:
# count distinct 'song_id'
df.select('song_id').distinct().count()

464805

In [6]:
df.groupBy('event').count().show()

+-----+--------+
|event|   count|
+-----+--------+
|    D|  635296|
|    S|  765613|
|    P|10816337|
+-----+--------+



In [7]:
df.select('uid', 'song_id').distinct().count()

3381777

### 1. generate frequency features based on play and download activities

In [8]:
df.select(F.min(F.col('date')).alias('start_date'),
         F.max(F.col('date')).alias('end_date')).show()

+----------+----------+
|start_date|  end_date|
+----------+----------+
|2017-03-30|2017-05-12|
+----------+----------+



In [9]:
import datetime
from dateutil import parser

recommend_time_window = 44
recommend_time_endtime = parser.parse('2017-05-12').date()
recommend_time_starttime = recommend_time_endtime - datetime.timedelta(recommend_time_window - 1)
print('recommendation time window days:', recommend_time_window,
     'that includes: ', recommend_time_starttime, '~~', recommend_time_endtime)

recommendation time window days: 44 that includes:  2017-03-30 ~~ 2017-05-12


In [10]:
def activity_frequency(df, event, time_window_list, snapshot_date):
    df_feature = df \
    .filter(F.col('event') == event) \
    .groupBy('uid', 'song_id') \
    .agg(*[F.sum(F.when((F.col('date')>=snapshot_date-datetime.timedelta(time_window-1)) &
                (F.col('date')<=snapshot_date), 1).otherwise(0)) \
           .alias('freq_' + event +'_last_'+str(time_window)) \
          for time_window in time_window_list])
    return df_feature

In [11]:
# ignore search activity
# use the whole time frame date

event_list = ['P', 'D']
time_window_list = [7,14,21,30,44]
snapshot_date = recommend_time_endtime
freq_feature_list = []

for event in event_list:
    freq_feature_list.append(activity_frequency(df, event, time_window_list, snapshot_date))

In [12]:
freq_feature_list

[DataFrame[uid: string, song_id: string, freq_P_last_7: bigint, freq_P_last_14: bigint, freq_P_last_21: bigint, freq_P_last_30: bigint, freq_P_last_44: bigint],
 DataFrame[uid: string, song_id: string, freq_D_last_7: bigint, freq_D_last_14: bigint, freq_D_last_21: bigint, freq_D_last_30: bigint, freq_D_last_44: bigint]]

In [13]:
#(freq_feature_list[0]).show(5)

In [14]:
pd.DataFrame((freq_feature_list[0]).take(5), columns=(freq_feature_list[0]).columns)

,uid,song_id,freq_P_last_7,freq_P_last_14,freq_P_last_21,freq_P_last_30,freq_P_last_44
0,168548493,1013317,0,0,0,0,1
1,168551450,3199100,0,0,0,0,1
2,168546317,7171098,0,0,0,0,1
3,168546877,77805,0,0,0,0,2
4,168552813,1148517,0,0,0,0,1


### 2. generate frequency feature based on play time

In [15]:
df_play = spark.read.csv('/Users/fanyang/Documents/musicbox/data/play_downsample.csv', header=True).cache()

In [16]:
pd.DataFrame(df_play.take(5), columns=df_play.columns)


,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,date
0,168551247,ar,11881432,0,ä¸æ¢¦æµ®ç,é¢æ·¡å¦å¦,78,149,0,2017-03-30
1,168551248,ip,21393368,0,éçï¼Introï¼,å¹¼ç¨å­ææ,87,87,0,2017-03-30
2,168550496,ar,12495422,0,æ¶å¥½ï¼ä½ ä»¬è¦çé«ä¸­æ°å­¦è¯¾,3Då¥å¦å¬è§é¦,369,2747,0,2017-03-30
3,168551179,ar,23485496,0,éå¡æ¶ä»£-(çµå½±ãéå¡ä»»å¡ãçå°¾æ²),åå¾åæ,26,212,0,2017-03-30
4,168548493,ip,6661513,0,å¹²ç©å¥³,å§è,63,243,0,2017-03-30


In [17]:
df_play.select('uid', 'song_id').distinct().count()

3123260

In [18]:
# df_play_select_1 : select 'uid' that had 'play_time' >0 and 'song_length'> 0

df_play_select_1 = df_play.filter((F.col('play_time')>0) & (F.col('song_length')>0))

In [19]:
# df_play_select2 : select 'uid' that 'play_time' <= 'song_length'

df_play_select_2 = df_play_select_1.filter(F.col('play_time') <= F.col('song_length'))

In [20]:
df_play_select_2.select('uid', 'song_id').distinct().count()

2005120

In [21]:
df_play_select_2['uid', 'song_id', 'play_time', 'song_length'].describe().show()

+-------+--------------------+--------------------+-----------------+-----------------+
|summary|                 uid|             song_id|        play_time|      song_length|
+-------+--------------------+--------------------+-----------------+-----------------+
|  count|             6683042|             6682142|          6683042|          6683042|
|   mean|1.6633839314438933E8|2.631828185012353...|943.3419000537052|262.7244700649295|
| stddev|1.5245499658298234E7|5.549839946838064E16|112455.4967262961|285.3364407901398|
|    min|           100071797|                  -1|                1|                1|
|    max|            99581051|             9999854|              999|              999|
+-------+--------------------+--------------------+-----------------+-----------------+



In [22]:
# count NA in each feature
df_play_select_2.select(*[F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df_play_select_2.columns]).show()


+---+------+-------+---------+---------+------+---------+-----------+---------+----+
|uid|device|song_id|song_type|song_name|singer|play_time|song_length|paid_flag|date|
+---+------+-------+---------+---------+------+---------+-----------+---------+----+
|  0|     0|    900|     6850|       19|  2145|        0|          0|        0|   0|
+---+------+-------+---------+---------+------+---------+-----------+---------+----+



In [23]:
# a song that has been played over 90% of song_length is considered to be a complete song 

def complete_play_freq(df, time_window_list, snapshot_date):
    df_feature = df \
        .groupBy('uid', 'song_id') \
        .agg(*[F.sum(F.when((F.col('date')>=snapshot_date-datetime.timedelta(time_window-1)) \
                            & (F.col('date')<=snapshot_date) \
                            & (F.col('play_time')/F.col('song_length')>=0.9),1).otherwise(0))\
               .alias('comp_play_last_'+str(time_window)) \
                for time_window in time_window_list])
    return df_feature

In [24]:
time_window_list = [7,14,21,30,44]
snapshot_date = recommend_time_endtime
cmp_play_freq_list = complete_play_freq(df_play_select_2, time_window_list, snapshot_date)

cmp_play_freq_list 

DataFrame[uid: string, song_id: string, comp_play_last_7: bigint, comp_play_last_14: bigint, comp_play_last_21: bigint, comp_play_last_30: bigint, comp_play_last_44: bigint]

In [25]:
pd.DataFrame(cmp_play_freq_list.take(5), columns=cmp_play_freq_list.columns)

,uid,song_id,comp_play_last_7,comp_play_last_14,comp_play_last_21,comp_play_last_30,comp_play_last_44
0,168546317,7171098,0,0,0,0,0
1,168546877,77805,0,0,0,0,2
2,168552813,1148517,0,0,0,0,0
3,168553236,20674146,0,0,0,1,2
4,168553209,16540515,0,0,0,0,0


### 3. combine all features into one dateframe
##### freq_feature_list: 'uid' , 'song_id' with frequency of play/download  ----> list
##### cmp_play_freq_list : 'uid', 'song_id' with frequency of completely played song ----> dataframe

In [26]:
df_model1 = cmp_play_freq_list.join(freq_feature_list[0], on=['uid', 'song_id'], how='left')

In [27]:
df_model2 = df_model1.join(freq_feature_list[1], on=['uid', 'song_id'], how='left')

In [28]:
df_model2.columns

['uid',
 'song_id',
 'comp_play_last_7',
 'comp_play_last_14',
 'comp_play_last_21',
 'comp_play_last_30',
 'comp_play_last_44',
 'freq_P_last_7',
 'freq_P_last_14',
 'freq_P_last_21',
 'freq_P_last_30',
 'freq_P_last_44',
 'freq_D_last_7',
 'freq_D_last_14',
 'freq_D_last_21',
 'freq_D_last_30',
 'freq_D_last_44']

In [29]:
# 'recommender_model01_0116.csv' this model contain songs that played over 90% of song_length
df_model2.fillna(0).toPandas().to_csv('/Users/fanyang/Documents/musicbox/data/recommender_model01_0116.csv')

### 4. generate features that contain songs has been play for half length

In [ ]:
# a song that has been played over 50% of song_length is considered to be a complete song 

def half_play_freq(df, time_window_list, snapshot_date):
    df_feature = df \
        .groupBy('uid', 'song_id') \
        .agg(*[F.sum(F.when((F.col('date')>=snapshot_date-datetime.timedelta(time_window-1)) \
                            & (F.col('date')<=snapshot_date) \
                            & (F.col('play_time')/F.col('song_length')>=0.5),1).otherwise(0))\
               .alias('half_play_last_'+str(time_window)) \
                for time_window in time_window_list])
    return df_feature

time_window_list = [7,14,21,30,44]
snapshot_date = recommend_time_endtime
half_play_freq_list = half_play_freq(df_play_select_2, time_window_list, snapshot_date)

In [ ]:
df_model3 = half_play_freq_list.join(freq_feature_list[0], on=['uid', 'song_id'], how='left')

In [ ]:
df_model4 = df_model3.join(freq_feature_list[1], on=['uid', 'song_id'], how='left')

In [ ]:
df_model4.columns

In [ ]:
# 'recommender_model_02h_0116.csv' this model contain songs that played over 50% of song_length
df_model4.fillna(0).toPandas().to_csv('/Users/fanyang/Documents/musicbox/data/recommender_model_02h_0116.csv')